# OpenAI Function Calling In LangChain

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from typing import List
from pydantic import BaseModel, Field

## Pydantic Syntax

Pydantic data classes are a blend of Python's data classes with the validation power of Pydantic. 

They offer a concise way to define data structures while ensuring that the data adheres to specified types and constraints.

In standard python you would create a class like this:

In [3]:
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email

In [4]:
foo = User(name="Joe",age=32, email="joe@gmail.com")

In [5]:
foo.name

'Joe'

In [6]:
foo = User(name="Joe",age="bar", email="joe@gmail.com")

In [7]:
foo.age

'bar'

In [8]:
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [9]:
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")

In [10]:
foo_p.name

'Jane'

<p style=\"background-color:#F5C780; padding:15px\"><b>Note:</b> The next line is expected to fail.</p>

In [11]:
foo_p = pUser(name="Jane", age="bar", email="jane@gmail.com")

ValidationError: 1 validation error for pUser
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='bar', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing

In [12]:
class Class(BaseModel):
    students: List[pUser]

In [13]:
obj = Class(
    students=[pUser(name="Jane", age=32, email="jane@gmail.com")]
)

In [14]:
obj

Class(students=[pUser(name='Jane', age=32, email='jane@gmail.com')])

## Pydantic to OpenAI function definition


In [15]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

In [16]:
from langchain_core.utils.function_calling import convert_to_openai_function

In [17]:
weather_function = convert_to_openai_function(WeatherSearch)

In [18]:
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [19]:
class WeatherSearch1(BaseModel):
    airport_code: str = Field(description="airport code to get weather for")

<p style=\"background-color:#F5C780; padding:15px\"><b>Note:</b> The next cell is expected to generate an error.</p>

In [20]:
convert_to_openai_function(WeatherSearch1)

{'name': 'WeatherSearch1',
 'description': '',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [21]:
class WeatherSearch2(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str

In [22]:
convert_to_openai_function(WeatherSearch2)

{'name': 'WeatherSearch2',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [23]:
from langchain_openai import ChatOpenAI

In [24]:
model = ChatOpenAI(model="gpt-5-mini")

In [25]:
model.invoke("what is the weather in SF today?", functions=[weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 149, 'total_tokens': 175, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CsuxAhsoY3GzQWPD5axnxFZbuyYES', 'service_tier': 'default', 'finish_reason': 'function_call', 'logprobs': None}, id='lc_run--019b759b-638b-7ac3-861c-feaad0db1f0c-0', usage_metadata={'input_tokens': 149, 'output_tokens': 26, 'total_tokens': 175, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [26]:
model_with_function = model.bind(functions=[weather_function])

In [27]:
model_with_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 148, 'total_tokens': 174, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CsuxDBdXfWjNHxSGXPOjNHkGtxcCN', 'service_tier': 'default', 'finish_reason': 'function_call', 'logprobs': None}, id='lc_run--019b759b-6c6f-7be0-bda6-108fcb739364-0', usage_metadata={'input_tokens': 148, 'output_tokens': 26, 'total_tokens': 174, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Forcing it to use a function

We can force the model to use a function

In [28]:
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name":"WeatherSearch"})

In [29]:
model_with_forced_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 148, 'total_tokens': 238, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CsuxEprZZ5FLdYQMbIou7xzbPEjok', 'service_tier': 'default', 'finish_reason': 'function_call', 'logprobs': None}, id='lc_run--019b759b-7259-7b62-9575-fd66406332d8-0', usage_metadata={'input_tokens': 148, 'output_tokens': 90, 'total_tokens': 238, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 64}})

In [30]:
model_with_forced_function.invoke("hi!")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"JFK"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 218, 'prompt_tokens': 143, 'total_tokens': 361, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CsuxGzruW4SUqXWcGvQL6K9PKXQp6', 'service_tier': 'default', 'finish_reason': 'function_call', 'logprobs': None}, id='lc_run--019b759b-78db-7990-b0fb-583a4fda067e-0', usage_metadata={'input_tokens': 143, 'output_tokens': 218, 'total_tokens': 361, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 192}})

## Using in a chain

We can use this model bound to function in a chain as we normally would

In [31]:
from langchain_core.prompts import ChatPromptTemplate

In [32]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])

In [33]:
chain = prompt | model_with_function

In [34]:
chain.invoke({"input": "what is the weather in sf?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 157, 'total_tokens': 183, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CsuxKyEuHoDDQTprYLUvqMHXtv2uL', 'service_tier': 'default', 'finish_reason': 'function_call', 'logprobs': None}, id='lc_run--019b759b-88ab-79c2-8fb8-ce57fa1dbefe-0', usage_metadata={'input_tokens': 157, 'output_tokens': 26, 'total_tokens': 183, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Using multiple functions

Even better, we can pass a set of function and let the LLM decide which to use based on the question context.

In [35]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")

In [36]:
functions = [
    convert_to_openai_function(WeatherSearch),
    convert_to_openai_function(ArtistSearch),
]

In [37]:
model_with_functions = model.bind(functions=functions)

In [38]:
model_with_functions.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 192, 'total_tokens': 282, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CsuxL3QMg5Jdc0ImauKVUZZkED3yp', 'service_tier': 'default', 'finish_reason': 'function_call', 'logprobs': None}, id='lc_run--019b759b-8e13-7e73-a120-518c563ce7b7-0', usage_metadata={'input_tokens': 192, 'output_tokens': 90, 'total_tokens': 282, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 64}})

In [39]:
model_with_functions.invoke("what are three songs by taylor swift?")

AIMessage(content='Here are three songs by Taylor Swift:\n\n- "Love Story"  \n- "Blank Space"  \n- "Shake It Off"', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 99, 'prompt_tokens': 194, 'total_tokens': 293, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CsuxNqw5VZwMSQZ6ZehP9ViKLTYgX', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b759b-96ea-7100-b02c-deb350fea801-0', usage_metadata={'input_tokens': 194, 'output_tokens': 99, 'total_tokens': 293, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 64}})

In [40]:
model_with_functions.invoke("hi!")

AIMessage(content='Hi — hi! How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 187, 'total_tokens': 207, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CsuxQwyuZ5aqQCyVJuwqMSQmc3Lqk', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b759b-a0fd-7de2-8979-5981c1b752e0-0', usage_metadata={'input_tokens': 187, 'output_tokens': 20, 'total_tokens': 207, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Generic tool execution (tool registry + executor)

Tool calling means the model can *propose* function calls. To actually run Python functions,
you need an execution step that:
- reads `tool_calls` from the model output
- dispatches to the corresponding tool
- returns `ToolMessage` results back to the model

This section implements a **generic** dispatcher (no hard-coded tool names).


In [41]:
from langchain_core.messages import ToolMessage
from langchain_core.runnables import RunnableLambda
from langchain_core.tools import tool, BaseTool
from langchain_openai import ChatOpenAI

@tool
def get_current_weather(location: str) -> str:
    """Get the current weather for a given location."""
    return f"The weather in {location} is sunny."

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two integers and return the product."""
    return a * b

tools = [get_current_weather, multiply]

llm = ChatOpenAI(model="gpt-5-mini", temperature=0).bind_tools(tools)


In [42]:
def build_tool_registry(tools: list[BaseTool]) -> dict[str, BaseTool]:
    return {t.name: t for t in tools}


def execute_tools_to_messages_factory(tool_registry: dict[str, BaseTool]):
    def execute_tools(ai_msg):
        tool_messages = []
        for call in ai_msg.tool_calls or []:
            tool_name = call["name"]
            tool_args = call["args"]
            tool_id = call["id"]

            print(f"[tool-exec] Invoking tool: {tool_name} with args: {tool_args}")

            tool_obj = tool_registry.get(tool_name)
            if tool_obj is None:
                raise ValueError(f"Model requested unknown tool: {tool_name}")

            result = tool_obj.invoke(tool_args)
            tool_messages.append(ToolMessage(content=str(result), tool_call_id=tool_id))
        return tool_messages

    return execute_tools


tool_registry = build_tool_registry(tools)
tool_step = RunnableLambda(execute_tools_to_messages_factory(tool_registry))


In [43]:
SYSTEM_TOOL_INSTRUCTIONS = (
    "You are a tool-using assistant. "
    "Use get_current_weather for weather questions and multiply for arithmetic. "
    "If the user asks multiple things, call ALL required tools before answering."
)

def run_full(question: str) -> str:
    # 1) model proposes tool calls
    ai = llm.invoke([("system", SYSTEM_TOOL_INSTRUCTIONS), ("user", question)])

    # 2) execute tools requested by the model
    tool_msgs = tool_step.invoke(ai)

    # 3) send tool outputs back to model for final response
    final = llm.invoke([("system", SYSTEM_TOOL_INSTRUCTIONS), ("user", question), ai, *tool_msgs])
    return final.content

run_full("Use tools: call get_current_weather for Athens and call multiply for 6 and 7. Then answer with both results.")


[tool-exec] Invoking tool: get_current_weather with args: {'location': 'Athens'}
[tool-exec] Invoking tool: multiply with args: {'a': 6, 'b': 7}


'Results:\n- Weather in Athens: sunny.\n- 6 × 7 = 42.'